# Language Modeling using Ngram

In this Exercise, you are going to use NLTK which is a natural language processing library for python to create a bigram language model and its variation. You will build one model for each of the following type and calculate their perplexity:
- Unigram Model
- Bigram Model
- Bigram Model with Laplace smoothing
- Bigram Model with Interpolation
- Bigram Model with Kneser-ney Interpolation
- Neural LM (optional)



In [1]:
#download corpus
!wget --no-check-certificate https://github.com/ekapolc/nlp_2019/raw/master/HW4/BEST2010.zip
!unzip BEST2010.zip

--2021-02-14 03:22:29--  https://github.com/ekapolc/nlp_2019/raw/master/HW4/BEST2010.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ekapolc/nlp_2019/master/HW4/BEST2010.zip [following]
--2021-02-14 03:22:29--  https://raw.githubusercontent.com/ekapolc/nlp_2019/master/HW4/BEST2010.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7423530 (7.1M) [application/zip]
Saving to: ‘BEST2010.zip’

BEST2010.zip        100%[===================>]   7.08M  19.3MB/s    in 0.4s    

2021-02-14 03:22:30 (19.3 MB/s) - ‘BEST2010.zip’ saved [7423530/7423530]

Archive:  BEST2010.zip
   creating: BEST2010/
  i

In [2]:
#First we import necessary library such as math, nltk, bigram, and collections.
import math
import nltk
import io
import random
from random import shuffle
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
random.seed(999)

BEST2010 is a free Thai NLP dataset by NECTEC usually use as a standard benchmark for various NLP tasks includeing language modeling. BEST2010 is separated into 4 domain article, encyclopedia, news and novel. The data is already  tokenized using '|' as a separator.

For example,

ตาม|ที่|นางประนอม ทองจันทร์| |กับ| |ด.ช.กิตติพงษ์ แหลมผักแว่น| |และ| |ด.ญ.กาญจนา กรองแก้ว| |ป่วย|สงสัย|ติด|เชื้อ|ไข้|ขณะ|นี้|ยัง|ไม่|ดี|ขึ้น|

In [6]:
# We choose news domain as our dataset
best2010=[]
fp= io.open('BEST2010/news.txt','r',encoding='utf-8')
for i,line in enumerate(fp):
    best2010.append(line.strip()[:-1])
fp.close()
all_vocabulary =set()
total_word_count =0
for line in best2010:
    for word in line.split('|'):        
        all_vocabulary.add(word)
        total_word_count+=1

In [7]:
#For simplicity, we assumes that each line is a sentence.
print ('Total sentences in BEST2010 news dataset :\t'+ str(len(best2010)))
print ('Total word counts in BEST2010 news dataset :\t'+ str(total_word_count))
print ('Total vocabulary in BEST2010 news dataset :\t'+ str(len(all_vocabulary)))

Total sentences in BEST2010 news dataset :	30969
Total word counts in BEST2010 news dataset :	1660190
Total vocabulary in BEST2010 news dataset :	35488


We separate out input into 2 sets, train and test data with 70:30 ratio

In [234]:
sentences = best2010
# The data is separated to train and test set with 70:30 ratio.
train = sentences[:int(len(sentences)*0.7)]
test = sentences[int(len(sentences)*0.7):]

#Training data
train_vocabulary =set()
train_word_count =0
for line in train:
    for word in line.split('|'):        
        train_vocabulary.add(word)
        train_word_count+=1
print ('Total sentences in BEST2010 news training dataset :\t'+ str(len(train)))
print ('Total word counts in BEST2010 news training dataset :\t'+ str(train_word_count))
print ('Total vocabuary in BEST2010 news training dataset :\t'+ str(len(train_vocabulary)))
# We will use 1/vocab_size as a default value for unknown word
unk_value = math.pow(len(train_vocabulary),-1)

Total sentences in BEST2010 news training dataset :	21678
Total word counts in BEST2010 news training dataset :	1042797
Total vocabuary in BEST2010 news training dataset :	26240


# Unigram

In this section, we will demonstrate how to build a unigram language model <br>
**Important note:** <br>
**\<s\>** = sentence start symbol <br>
**\</s\>** = sentence end symbol 

In [ ]:
def getUnigramModel(data):
    model = defaultdict(lambda: 0)
    word_count =0
    for sentence in data:
        sentence +=  u'|</s>' #for unigram model we can always ignore <s>, since p(w0=<s>)=1
        for w1 in sentence.split('|'):
            model[w1] +=1.0
            word_count+=1
    for w1 in model:
        model[w1] = model[w1]/(word_count)
    return model

In [ ]:
model = getUnigramModel(train)

In [233]:
def getLnValue(x):
    if x >0.0:
        return math.log(x)
    else:
        return math.log(unk_value)

In [ ]:
#problability of 'นายก'
print(getLnValue(model[u'นายก']))
#for example, problability of 'นายกรัฐมนตรี' which is an unknown word is equal to
print(getLnValue(model[u'นายกรัฐมนตรี']))
#problability of 'นายก' 'ได้' 'ให้' 'สัมภาษณ์' 'กับ' 'สื่อ'
prob = getLnValue(model[u'นายก'])+getLnValue(model[u'ได้'])+ getLnValue(model[u'ให้'])+getLnValue(model[u'สัมภาษณ์'])+getLnValue(model[u'กับ'])+getLnValue(model[u'สื่อ'])+getLnValue(model['</s>'])
print('Ln', prob)
print ('Problability of a sentence', math.exp(prob))

-6.551526663995246
-10.175040243058024
Ln -39.72069644125504
Problability of a sentence 5.617210748667918e-18


# Perplexity

In order to compare language model we need to calculate perplexity. In this task you should write a perplexity calculation code for the unigram model. The result perplexity should be around 556.39 and
476.07 on train and test data.

# TODO #0
**Fill your name and ID here** <br>
**Name**: Phatcharapon Jumruspun
<br>
**ID**: 6031040921

## TODO #1 **Calculate perplexity**

In [ ]:
def calculate_sentence_ln_prob(sentence, model):
  sum = 0
  words = sentence.split('|')
  for word in words:
    # print(getLnValue(model[word]))
    sum += getLnValue(model[word])
  sum+= getLnValue(model['</s>'])
  return sum

def perplexity(test,model):
  sum = 0
  word_cnt = 0
  for line in test:
    sentence = line.split('|')
    word_cnt += len(sentence)+1
    sum += calculate_sentence_ln_prob(line, model)
  exp_factor = -sum/word_cnt
  return math.exp(exp_factor)

In [ ]:
print(perplexity(train,model))
print(perplexity(test,model))

556.3925994212195
476.0687892303532


## TODO #2 **Please explain why this model give us such a high perplexity.**

**Your answer**:  
Unigram model doesn't really represent the probability of the sentences occurring in the language. As it only represents the probability of word found in the language alone. It lacks the "Context" around each word in sentences.

# Bigram

Next, you will create a better language model than a unigram (which is not much to compare with). But first, it is very tedious to count every pair of words that occur in our corpus by ourselves. In this case, nltk provide us a simple library which will do it for us.

In [ ]:
#example of nltk usage for bigram
sentence = 'I always search google for an answer .'

print('This is how nltk generate bigram.')
for w1,w2 in bigrams(sentence.split(), pad_right=True, pad_left=True):
    print (w1,w2)
print('None is used as a start and end of sentence symbol.')

This is how nltk generate bigram.
None I
I always
always search
search google
google for
for an
an answer
answer .
. None
None is used as a start and end of sentence symbol.


Now, you should be able to implement a bigram model by yourself. Also, you must create a new perplexity calculation for bigram. The result perplexity should be around 58.78 and 146.26 on train and test data.

## TODO #3 **Create a Bigram Model**

In [139]:
def getBigramModel(data):
  ###FILL YOUR CODE HERE###
  unigram_count = defaultdict(lambda: 0)
  bigram_count = defaultdict(lambda: 0)

  for sentence in data:
    for w1, w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
      w1 = str(w1)
      w2 = str(w2)
      if w1 == 'None': w1 = '<s>'
      if w2 == 'None': w2 = '</s>'
      unigram_count[w1] += 1
      bigram_count[(w1, w2)] += 1

  bigram_prob = defaultdict(lambda: 0)

  for sentence in data: 
    for w1, w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
      w1 = str(w1)
      w2 = str(w2)
      if w1 == 'None': w1 = '<s>'
      if w2 == 'None': w2 = '</s>'
      bigram_prob[(w1, w2)] = bigram_count[(w1, w2)]/unigram_count[w1]

  return bigram_prob

model = getBigramModel(train)

In [140]:
model[('สงสัย','ว่า')]

0.12128712871287128

## TODO #4 **Calculate Perplexity for Bigram Model**



In [172]:
def calculate_sentence_ln_prob(sentence, model,mode='1D'):
  sum = 0
  for w1, w2 in bigrams(sentence.split('|'), pad_left=True, pad_right=True):
    if w1 == None: w1 = '<s>'
    if w2 == None: w2 = '</s>'
    if mode == '1D':
      # print(w1, w2, getLnValue(model[(w1, w2)]))
      sum += getLnValue(model[(w1, w2)])
    elif mode == '2D':
      # print(w1, w2, getLnValue(model[w1][w2]))
      sum += getLnValue(model[w1][w2])
  return sum

def perplexity(test,model,mode='1D'):
  sum = 0
  word_cnt = 0
  for line in test:
    sentence = line.split('|')
    word_cnt += len(sentence)+1
    sum += calculate_sentence_ln_prob(line, model, mode)
  exp_factor = -sum/word_cnt
  return math.exp(exp_factor)

In [143]:
print (perplexity(train,model) )
print (perplexity(test, model))

# 58.78942889767147
# 146.26539331038614

58.78942889767147
146.26539331038614


# Smoothing

Usually any ngram models have a sparsity problem, which means it does not have every possible ngram of words in the dataset. Smoothing techniques can alleviate this problem. In this section, you will implement two basic smoothing methods laplace smoothing and interpolation for bigram.

## TODO #5 **Bigram with Laplace smoothing (Add-One Smoothing)**

In [123]:
def getBigramWithLaplaceSmoothing(data):
  unigram_count = defaultdict(lambda: 0)
  bigram_count = defaultdict(lambda: 0)

  for sentence in data:
    for w1, w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
      w1 = str(w1)
      w2 = str(w2)
      if w1 == 'None': w1 = '<s>'
      if w2 == 'None': w2 = '</s>'
      unigram_count[w1] += 1
      bigram_count[(w1, w2)] += 1

  laplace = defaultdict(lambda: 0)

  for sentence in data: 
    for w1, w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
      w1 = str(w1)
      w2 = str(w2)
      if w1 == 'None': w1 = '<s>'
      if w2 == 'None': w2 = '</s>'
      laplace[(w1, w2)] = (bigram_count[(w1, w2)]+1)/(unigram_count[w1]+len(train_vocabulary))

  return laplace

model = getBigramWithLaplaceSmoothing(train)

In [121]:
model[('<s>', 'การ')]

0.004215534872073125

In [124]:
print (perplexity(train,model))
print (perplexity(test, model))

# 974.8134581679766
# 1098.1622194979489

974.7840907514313
1098.1361205594044


## TODO #6 **Bigram with Interpolation**
lambda value is 0.7 for bigram, 0.25 for unigram, and 0.05 for unknown word

In [196]:
class key_dependent_dict(defaultdict):
  def __init__(self, key_func):
    super().__init__(None)
    self.key_func = key_func
  def __missing__(self, key):
    res = self.key_func(key)
    self[key] = res
    return res

In [269]:
#interpolation

def getBigramWithInterpolation(data,unk=unk_value,handle_oov='unk'):
    ###FILL YOUR CODE HERE###
  expected_oov_handling = ['unk', 'unigram_unk']
  if handle_oov not in expected_oov_handling:
    raise Exception(f"handle_oov parameter incorrect, expected 1 of '{expected_oov_handling}' but got '{handle_oov}'")
    return

  unigram_count = defaultdict(lambda: 0)
  bigram_count = defaultdict(lambda: 0)

  for sentence in data:
    for w1, w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
      w1 = str(w1)
      w2 = str(w2)
      if w1 == 'None': w1 = '<s>'
      if w2 == 'None': 
        w2 = '</s>'
        unigram_count[w2] += 1
      unigram_count[w1] += 1
      bigram_count[(w1, w2)] += 1
  
  unique_vocab = len(unigram_count)
  unigram_sum = sum(unigram_count.values())

  if unk == 'count': unk = 1/unique_vocab

  if handle_oov == expected_oov_handling[1]:
    def getValueInterpolation(w1, w2):
      if unigram_count[w1] == 0:
        return 0
      else:
        return 0.25 * unigram_count[w2]/unique_vocab + 0.05 * unk
    inter = key_dependent_dict(lambda args: getValueInterpolation(args[0], args[1]))

  if handle_oov == expected_oov_handling[0]:
    inter = defaultdict(lambda: 0)

  for (w1, w2) in bigram_count.keys():
    w1 = str(w1)
    w2 = str(w2)
    if w1 == 'None': w1 = '<s>'
    if w2 == 'None': w2 = '</s>'
    lambda2 = 0.7 * bigram_count[(w1, w2)]/unigram_count[w1]
    lambda1 = 0.25 * unigram_count[w2]/unigram_sum
    lambda0 = 0.05 * unk
    inter[(w1, w2)] = lambda2 + lambda1 + lambda0

  return inter

In [274]:
model = getBigramWithInterpolation(train)
print (perplexity(train,model))        
print (perplexity(test,model))

# 73.38409869825665
# 172.67485908813356

73.69857239963139
173.39908315559052


In [271]:
# When bigram not found but unigram found: return unigram + unk
# Instead of just returning unk alone

model = getBigramWithInterpolation(train, handle_oov='unigram_unk')
print (perplexity(train,model))        
print (perplexity(test,model))

73.69857239963139
114.01798249183278


the result perplexity on training and testing should be 

    974.81, 1098.16 for Laplace smoothing
    73.38, 172.67 for Interpolation

# Language modeling on multiple domains

Sometimes, we do not have enough data to create a language model for a new domain. In that case, we can improvised by combining several models to improve result on the new domain.

In this exercise you will try to merge two language models from news and article domains to create a language model for the encyclopedia domain.

In [272]:
# create article data
encyclo_data=[]
fp= io.open('BEST2010/encyclopedia.txt','r',encoding='utf-8')
for i,line in enumerate(fp):
    encyclo_data.append(line.strip()[:-1])
fp.close()

First, you should try to calculate perplexity of your bigram with interpolation using "news data" (train) on "encyclopedia data" (test). The result perplexity should be around 727.35.

For your information, a bigram model with interpolation using "ariticle data" (train) to test on "encyclopedia data" (test) has a perplexity of 505.79.

In [275]:
# print perplexity of bigram with interpolation on article data        
# 727.3502637212223
print (perplexity(encyclo_data,model))

730.3982765453854


## TODO #7 
Write a model that produce 450.0 or less perplexity on encyclopedia data without using data from the encyclopedia as training data. (Hint : Try to combine a model with news data and a model with article data together.)

In [279]:
# Fill code here
news = getBigramWithInterpolation(train)
article = getBigramWithInterpolation(encyclo_data)

combined_model = dict(news)
combined_model.update(article)
# 428.85251789073953 (on combined data)
print('Perplexity of combine Bigram model with interpolation smoothing on encyclopedia test data',perplexity(encyclo_data, combined_model))

Perplexity of combine Bigram model with interpolation smoothing on encyclopedia test data 82.67199860229121


## TODO #8 
## Kneser-ney on "News"

<!-- Reimplement equation 4.33 in SLP textbook (https://lagunita.stanford.edu/c4x/Engineering/CS-224N/asset/slp4.pdf) -->

Implement Bigram Knerser-ney LM. The result perplexity should be around 55.06 and 140.76 on train and test data. 


In [293]:
# Fill codehere
discount = 0.75

def getKnModel(data, handle_oov='unk'):
  expected_oov_handling = ['unk', 'pcon']
  if handle_oov not in expected_oov_handling:
    raise Exception(f"handle_oov parameter incorrect, expected 1 of '{expected_oov_handling}' but got '{handle_oov}'")
    return

  unigram_count = defaultdict(lambda: 0)
  bigram_count = defaultdict(lambda: 0)

  for sentence in data:
    for w1, w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
      w1 = str(w1)
      w2 = str(w2)
      if w1 == 'None': w1 = '<s>'
      if w2 == 'None': w2 = '</s>'
      unigram_count[w1] += 1
      bigram_count[(w1, w2)] += 1

  vertical_unique = defaultdict(lambda: 0)
  horizontal_unique = defaultdict(lambda: 0)
  unique = len(bigram_count)

  for (w1, w2), count in bigram_count.items():
    horizontal_unique[w1] += 1
    vertical_unique[w2] += 1

  if handle_oov == expected_oov_handling[1]:
    def getValuePKN(w1, w2):
      if unigram_count[w1] == 0:
        return 0
      else:
        # OOV
        l = discount*horizontal_unique[w1]/unigram_count[w1]
        pcon = vertical_unique[w2]/unique
        return l*pcon
    pkn2 = key_dependent_dict(lambda args: getValuePKN(args[0], args[1]))
  elif handle_oov == expected_oov_handling[0]:
    pkn2 = defaultdict(lambda: 0)

  for sentence in data: 
    for w1, w2 in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
      w1 = str(w1)
      w2 = str(w2)
      if w1 == 'None': w1 = '<s>'
      if w2 == 'None': w2 = '</s>'
      mx = max(bigram_count[(w1, w2)]-discount, 0)/unigram_count[w1]
      l = discount*horizontal_unique[w1]/unigram_count[w1]
      pcon = vertical_unique[w2]/unique
      pkn2[(w1, w2)] = mx+l*pcon

  return pkn2

In [301]:
debug = [
  'I|am|Sam',
  'Sam|I|am',
  'I|am|Sam',
  'I|like|green|eggs'
]
model = getKnModel(debug)
model_pcon = getKnModel(debug, handle_oov='pcon')
print([
  model[('<s>','am')],
  model[('am','Sam')],
  model[('Sam','</s>')],
])
print([
  model_pcon[('<s>','am')],
  model_pcon[('am','Sam')],
  model_pcon[('Sam','</s>')],
])

[0, 0.5075757575757576, 0.553030303030303]
[0.03409090909090909, 0.5075757575757576, 0.553030303030303]


In [302]:
# perplexity(['am|Sam'], model)
calculate_sentence_ln_prob('am|Sam', model_pcon)

-4.649176310443481

In [303]:
perplexity(['am|Sam'], model_pcon)
# In slides ~ 4.71 as well

4.710176763899694

In [299]:
model = getKnModel(train)
print (perplexity(train,model))        
print (perplexity(test,model))

# 71.14054002208687
# 174.02464248000433
# New test perplex from Ajarn: ~142, ~155?
# Not found in bigram -> unk = 155

71.14054002208687
155.09274968738495


In [304]:
model_pcon = getKnModel(train, handle_oov='pcon')
print (perplexity(train,model_pcon))        
print (perplexity(test,model_pcon))

# Not found in bigram -> weight * pcon = 142

71.14054002208687
142.99781709085792


## TODO #9
## Neural LM (optional)
do it on news corpus that we splitted into train and test sets at the beginning of this exercise. 

In [ ]:
#there are many ways to do this. e.g.:
#https://machinelearningmastery.com/develop-word-based-neural-language-models-python-keras/